<a href="https://colab.research.google.com/github/CarolMartinezPerez/03MAIR-Algoritmos-de-Optimizacion-2019/blob/master/RL/Introduccio%CC%81n_al_aprendizaje_por_refuerzo_Ejercicio_de_evaluacio%CC%81n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Teoría

- Esta parte será el 40% de la nota final del bloque de Aprendizaje por Refuerzo.

Define brevemente qué es el aprendizaje por refuerzo. ¿Qué diferencias hay entre aprendizaje supervisado, no supervisado y por refuerzo?

La diferencia es principalmente que el **Aprendizaje No supervisado**, a diferencia de los otros, no tiene información real del problema, ofrece una solución de identificación de patrones y procesos. En el **Aprendizaje Supervisado** la salida es conocida, y se entrena al sistema para enfrentarse a nuevas entradas. En el **Aprendizaje por Refuerzo** la respuesta no aparece normalmente como única, o al menos no el conjunto de respuestas, sino que trata de escoger la idónea según una valoración obtenida para llegar a un determinado objetivo global.



Según lo comentado en clase se podría tener en cuenta también como la siguiente diferenciación:

 - A. No supervisado -> descriptivo
 - A. Supervisado -> productivo
 - A. por refuerzo -> prescriptivo

Define con tus palabras los conceptos de Entorno, Agente, Recompensa, Estado y Observación.

- **Entorno**: Contexto en el que el agente debe escoger una respuesta o acción
- **Agente**: Ente que analiza la información e intenta escoger la mejor respuesta o acción
- **Recompensa**: Valoración a una determinada acción o respuesta
- **Estado**: Momento de decisión del agente sobre qué acción o respuesta dar para pasar al siguiente estado
- **Observación**: Valor de cada parámetro que constituye el entorno en un instante dado

Dependiendo del algoritmo de aprendizaje por refuerzo que se use, ¿qué clasificaciones podemos encontrar? Coméntalas brevemente.

Lista tres diferencias entre los algoritmos de DQN y Policy Gradient

### Práctica

- Esta parte será el 60% de la nota final del bloque de Aprendizaje por Refuerzo.

Algunas consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, una solución óptima será alcanzar una media de recompensa por encima de 16 puntos. Para medir si hemos conseguido llegar a la solución óptima, la media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este bloque práctico consta de tres partes:

   1) Implementar la red neuronal que se usará en la solución
    
   2) Seleccionar los hiperparámetros adecuados para las distintas piezas de la solución DQN
    
   3) Justificar la respuesta en relación a los resultados obtenidos

IMPORTANTE:

- Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.

- Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).

- Si usáis Google Colab, recordad usar las versiones de Tensorflow==1.13.1, Keras==2.2.4 y keras-rl==0.4.2

In [0]:
# Uncomment this line for installing keras-rl on Google collaboratory
# !pip install keras-rl

In [0]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [0]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4


env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [0]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1) Implementación de la red neuronal

In [0]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=input_shape))

# TODO

model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

2) Selección de hiperparámetros para la solución DQN

In [0]:
# TODO - Select the parameters for the memory
memory = SequentialMemory()
processor = AtariProcessor()

In [0]:
# TODO - Select the parameters for the policy
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps')

In [0]:
# TODO - Select the parameters for the Agent and the Optimizer
dqn = DQNAgent()
dqn.compile(Adam(), metrics=['mae'])

In [0]:
# Training part
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

# TODO - Select the parameters for the method "fit"
dqn.fit()

dqn.save_weights(weights_filename, overwrite=True)

In [0]:
# Testing part to calculate the mean reward
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

3) Justificación de los parámetros seleccionados y de los resultados obtenidos